In [61]:
from skimpy import skim as sk
import json 
import requests 
import pandas as pd
from pandas import json_normalize 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim


In [62]:
df_degradations =  pd.read_csv('D:\\Top_gun\\Datasets\\df_degradations\\degradations_2024-06-10.csv')
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,54.443290,2024-06-06,False
1,D009402,E170_6353,REAE170-E170_6353-1,51.297412,2024-06-06,False
2,D000812,E170_6353,SYSE170-E170_6353-2,17.647642,2024-06-06,False
3,D006665,E170_6353,ORDE170-E170_6353-3,7.561442,2024-06-06,False
4,D001163,E170_6353,AUTE170-E170_6353-4,43.325214,2024-06-06,False


In [63]:
df_degradations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21060 entries, 0 to 21059
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ref_deg           21060 non-null  object 
 1   linked_aero       21060 non-null  object 
 2   compo_concerned   21060 non-null  object 
 3   usure_nouvelle    21060 non-null  float64
 4   measure_day       21060 non-null  object 
 5   need_replacement  21060 non-null  bool   
dtypes: bool(1), float64(1), object(4)
memory usage: 843.4+ KB


In [64]:
df_degradations['measure_day'].value_counts()

measure_day
2024-06-10             6525
2024-06-05             4950
2024-06-06             2700
2024-06-04             1755
2024-06-09             1485
2024-06-08             1035
2024-06-01              675
2024-06-02              540
2024-06-03              540
2024-06-07              495
2023-08-23 00:00:00     270
2024-01-21 00:00:00      45
2024-05-31               45
Name: count, dtype: int64

In [65]:
# Fonctions pour la Normalisation des données et preparation des données pour le model

def get_degradation_data(data):
    df_degradations['measure_day'] = pd.to_datetime(df_degradations['measure_day'], format='%Y-%m-%d', errors='coerce')
    pd.set_option('future.no_silent_downcasting', True)
    df_degradations['need_replacement'] = df_degradations['need_replacement'].replace({True: 1, False: 0})
    df_degradations['usure_nouvelle'] = round(df_degradations['usure_nouvelle'], 0)
    
get_degradation_data(df_degradations)    

In [66]:
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,54.0,2024-06-06,0
1,D009402,E170_6353,REAE170-E170_6353-1,51.0,2024-06-06,0
2,D000812,E170_6353,SYSE170-E170_6353-2,18.0,2024-06-06,0
3,D006665,E170_6353,ORDE170-E170_6353-3,8.0,2024-06-06,0
4,D001163,E170_6353,AUTE170-E170_6353-4,43.0,2024-06-06,0


In [67]:
skim(df_degradations)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 21060  │ │ string      │ 3     │                                                          │
│ │ Number of columns │ 6      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 1     │                                                          │
│                                │ int32       │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA   ┃ NA %   ┃ mean    ┃ sd      ┃ p0   ┃ p25   ┃ p50   ┃ p75  ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ usure_nouvelle         │    0 │      0 │   32.85 │   17.44 │    0 │    18 │    33 │   48 │    89 │  ▆▇▇▇▂  │  │
│ │ need_replacement       │    0 │      0 │       0 │       0 │    0 │     0 │     0 │    0 │     0 │     ▇   │  │
│ └────────────────────────┴──────┴────────┴─────────┴─────────┴──────┴───────┴───────┴──────┴───────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name           ┃ NA      ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ measure_day           │     315 │       1.5 │     2024-05-31      │     2024-06-10      │ None             │  │
│ └───────────────────────┴─────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                     ┃ NA     ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ref_deg                         │      0 │          0 │                          1 │                 21060 │  │
│ │ linked_aero                     │      0 │          0 │                          1 │                 21060 │  │
│ │ compo_concerned                 │      0 │          0 │                          1 │                 21060 │  │
│ └─────────────────────────────────┴────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [68]:
# Enregistrement du fichier degradations dans un fichier csv
df_degradations.to_csv('D:\\Top_gun\\Data_clean\\degradations_full_cleaned.csv', index=False)